# JSON Editor


In [22]:
import json
from pathlib import Path
from typing import List, Dict


file_path = 'FewShots_results(1)'
# 
# def load_experiment_results(results_dirs: List[str]) -> List[Dict]:
#     results = []
#     for results_dir in results_dirs:
#         for result_file in Path(results_dir).rglob('*.json'):
#             with open(result_file, 'r') as f:
#                 results.append(json.load(f))
#     return results
# 
# # results_dirs = []
# for i in range(1,4):
#     results_dirs.append(f'{file_path}/Run{i}')

import json
from pathlib import Path
from typing import List, Dict

file_path = 'FewShots_results(1)'

def update_experiment_results(results_dirs: List[str]) -> List[Dict]:
    results = []
    for results_dir in results_dirs:
        for result_file in Path(results_dir).rglob('*.json'):
            with open(result_file, 'r') as f:
                data = json.load(f)
                trunc_value = data.get('trunc', 50)  # Default to 50 if 'trunc' is not found
                data['description'] = f"{data['description']}_{trunc_value}"
                with open(result_file, 'w') as f_out:
                    json.dump(data, f_out, indent=4)
                results.append(data)
    return results

results_dirs = []
for i in range(1, 4):
    results_dirs.append(f'{file_path}/Run{i}')

results = update_experiment_results(results_dirs)
print(results)
# results = []
# for results_dir in results_dirs:
#     for result_file in Path(results_dir).rglob('*.json'):
#         if str(result_file).endswith('6.json'):
#             result_file.unlink()
#         else:
#             no_json = str(result_file)[:-5]
#             new = Path(no_json + "_trunc100" + ".json")
#             result_file.rename(new)

[{'description': 'few_shot_base_torch.float16_2_100', 'accuracy': 0.831, 'unknown': 0.0, 'prediction_ratio': 0.5974, 'actual_ratio': 0.5096, 'begin_time': 1731172556.0832539, 'end_time': 1731172942.18379, 'total_time': 386.10053610801697, 'trunc': 100}, {'description': 'few_shot_base_torch.float32_2_100', 'accuracy': 0.8342, 'unknown': 0.0, 'prediction_ratio': 0.5928, 'actual_ratio': 0.5062, 'begin_time': 1731181055.1310196, 'end_time': 1731181395.2256243, 'total_time': 340.0946047306061, 'trunc': 100}, {'description': 'few_shot_large_torch.float32_2_100', 'accuracy': 0.8868, 'unknown': 0.0, 'prediction_ratio': 0.5336, 'actual_ratio': 0.5024, 'begin_time': 1731182781.4587796, 'end_time': 1731190818.0575004, 'total_time': 8036.598720788956, 'trunc': 100}, {'description': 'few_shot_small_torch.float16_4_50', 'accuracy': 0.7268, 'unknown': 0.0, 'prediction_ratio': 0.6256, 'actual_ratio': 0.5028, 'begin_time': 1731533678.6405225, 'end_time': 1731533780.7194922, 'total_time': 102.0789697170

In [25]:
import json
from pathlib import Path
from typing import List, Dict
import pandas as pd

def load_experiment_results(results_dirs: List[str]) -> pd.DataFrame:
    results = []
    for results_dir in results_dirs:
        for result_file in Path(results_dir).rglob('*.json'):
            if 'result_replication' in result_file.name:
                with open(result_file, 'r') as f:
                    results.append(json.load(f))

    df = pd.DataFrame(results)
    df = df.join(pd.json_normalize(df.pop(df.columns[0])))

    return df

def process_all_folders_in_directory(directory: str) -> pd.DataFrame:
    all_dfs = []
    for folder in Path(directory).iterdir():
        if folder.is_dir():
            results_dirs = [str(folder)]
            df = load_experiment_results(results_dirs)
            all_dfs.append(df)
    big_df = pd.concat(all_dfs, ignore_index=True)
    return big_df

directory = 'exercise-3_firstTry'
big_dataframe = process_all_folders_in_directory(directory)

print(big_dataframe)

   accuracy  unknown  prediction_ratio  actual_ratio    begin_time  \
0      0.73      0.0              0.64          0.51  1.731430e+09   
1      0.00      1.0               NaN          0.54  1.731430e+09   
2      0.00      1.0               NaN          0.51  1.731431e+09   
3      0.00      1.0               NaN          0.51  1.731430e+09   
4      0.00      1.0               NaN          0.52  1.731430e+09   
5      0.00      1.0               NaN          0.43  1.731430e+09   
6      0.00      1.0               NaN          0.50  1.731430e+09   
7      0.77      0.0              0.60          0.45  1.731430e+09   

       end_time  total_time  rank  alpha    modules  
0  1.731430e+09   43.968657     8     16     [k, q]  
1  1.731430e+09   44.168845     8     32  [k, q, v]  
2  1.731431e+09   52.288482    16     32     [k, q]  
3  1.731430e+09   44.130644    16     16  [k, q, v]  
4  1.731430e+09   43.670245    16     16  [k, q, v]  
5  1.731431e+09  115.590908    16     32     

/var/folders/v9/n514v4yd4hn8vqwrpj4cdt8c0000gn/T/ipykernel_8169/3202665903.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  big_df = pd.concat(all_dfs, ignore_index=True)


In [26]:
import os
from pathlib import Path

def rename_files_in_directory(directory: str):
    for file_path in Path(directory).rglob('*torch.float16*'):
        new_file_path = str(file_path).replace('torch.float16', 'torch.float32')
        os.rename(file_path, new_file_path)
        print(f'Renamed: {file_path} to {new_file_path}')

directory = 'exercise-3_firstTry'
rename_files_in_directory(directory)

Renamed: exercise-3_firstTry/exp_0_0_rank=8_alpha=16_dtype=torch.float16_epochs=2 to exercise-3_firstTry/exp_0_0_rank=8_alpha=16_dtype=torch.float32_epochs=2
Renamed: exercise-3_firstTry/exp_1_1_rank=8_alpha=32_dtype=torch.float16_epochs=2 to exercise-3_firstTry/exp_1_1_rank=8_alpha=32_dtype=torch.float32_epochs=2
Renamed: exercise-3_firstTry/exp_1_3_rank=16_alpha=32_dtype=torch.float16_epochs=2 to exercise-3_firstTry/exp_1_3_rank=16_alpha=32_dtype=torch.float32_epochs=2
Renamed: exercise-3_firstTry/exp_0_2_rank=16_alpha=16_dtype=torch.float16_epochs=2 to exercise-3_firstTry/exp_0_2_rank=16_alpha=16_dtype=torch.float32_epochs=2
Renamed: exercise-3_firstTry/exp_1_2_rank=16_alpha=16_dtype=torch.float16_epochs=2 to exercise-3_firstTry/exp_1_2_rank=16_alpha=16_dtype=torch.float32_epochs=2
Renamed: exercise-3_firstTry/exp_0_3_rank=16_alpha=32_dtype=torch.float16_epochs=2 to exercise-3_firstTry/exp_0_3_rank=16_alpha=32_dtype=torch.float32_epochs=2
Renamed: exercise-3_firstTry/exp_0_1_rank=8_